In [1]:
# import necessary packages
import pandas as pd
from datetime import datetime

#### Fund Managers

In [2]:
# loading data of fund managers
# data source: iFind
# simple processing with Excel has been done
data_managers_raw = pd.read_excel(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Exceled\ADA_Data_Fund_Manager.xlsx")

In [3]:
# copy first
data_managers = data_managers_raw.copy()

# nominal to numerical
data_managers['manager_edu_num'] = [0 if i == '本科' else (1 if i == '硕士' else 2)for i in data_managers['manager_edu']]  
data_managers['female'] = [0 if i == '男' else 1 for i in data_managers['manager_gender']]  # nominal to numerical

# transform and calculate working experience(days)
end = datetime(2024, 12, 31)
data_managers['experience_days'] = [(end - datetime.strptime(i, "%Y-%m-%d")).days for i in data_managers['working_date']]

# obtain numbers of funds each manager is managing from the funds list
data_managers['num_fund_manager_curr'] = [len(i.split(','))for i in data_managers['funds_on']]

In [4]:
# select columns and rearrange
data_managers_output = data_managers[['fund_code', 'manager_name', 'female', 'manager_edu_num', 'experience_days', 'aum_manager', 'num_fund_manager_curr', 'num_fund_manager_hist']]

# save processed data as csv file
data_managers_output.to_csv(r'C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Processed\ADA_Data_Fund_Manager.csv', encoding='gbk')

In [5]:
# viewing data
data_managers_output.head()

,fund_code,manager_name,female,manager_edu_num,experience_days,aum_manager,num_fund_manager_curr,num_fund_manager_hist
0,000001.OF,刘睿聪,0,1,734,2.601916e+09,2,4
1,000003.OF,梅寓寒,1,1,1214,9.371041e+08,5,5
2,000004.OF,梅寓寒,1,1,1214,9.371041e+08,5,5
3,000005.OF,吴翠,1,1,420,4.541464e+09,4,4
4,000006.OF,盛丰衍,0,1,2149,7.756340e+09,9,15


#### Fund Companies

In [6]:
# loading data of fund companies
# data source: iFind
# simple processing with Excel has been done
data_companies_raw = pd.read_excel(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Exceled\ADA_Data_Fund_Company.xlsx")

In [7]:
# copy first
data_companies = data_companies_raw.copy()

# transform and calculate duration of companies
data_companies['company_duration'] = [(end - datetime.strptime(i, "%Y-%m-%d")).days for i in data_companies['company_starting_date']]

# get a rank of company (dtype: int)
data_companies['company_rank_aum'] = [int(i[:-4]) for i in data_companies['rank_aum']]

# sort by ranks of companies
data_companies.sort_values(by="company_rank_aum", ascending=True, inplace=True)

In [8]:
# select columns and rearrange
data_companies_output = data_companies[['company_name', 'company_name_short', 'company_rank_aum','aum_non_mmf', 'aum', 'num_funds',  'num_managers', 'avg_aum_manager', 'avg_num_manager', 'company_duration']]

# reset index 
data_companies_output.reset_index(inplace=True, drop=True)

# save processed data of fund companies
data_companies_output.to_csv(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Processed\ADA_Data_Fund_Company.csv", encoding='gbk')

#### Funds

In [9]:
# loading data of fund companies
# data source: iFind
# simple processing with Excel has been done
data_funds_raw = pd.read_excel(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Exceled\ADA_Data_Fund.xlsx")

In [10]:
# copy first
data_funds = data_funds_raw.copy()

In [11]:
# select necessary columns
# using data of 2024
data_funds_2024 = data_funds[['fund_code', 'fund_name', 'company_name_short', 'manager_name', 'manager_days', 'fof', 'index', 
        'category_l1', 'category_l2', 'nav', 'fee_management', 'fee_custodian', 'fee_subscription_max', 'fee_redemption_max ', 
        'ror_2024', 'pct_stock_2024', 'pct_bond_2024', 'pct_fund_2024', 'pct_mm_2024', 'pct_deposit_2024', 'fund_inception_date']]

In [12]:
# transform the format of date
data_funds_2024.loc[:, 'fund_inception_date'] = [datetime.strptime(i, '%Y-%m-%d') for i in data_funds_2024['fund_inception_date']]

# filter the data
start_2024 = datetime(2024, 1, 1)
data_funds_2024 = data_funds_2024[
                                (data_funds_2024['manager_days'] >= 375)  # fund under specific manager's management since 2024-01-01   
                                & (data_funds_2024['fof'] == '否')  # remove FoFs
                                & (data_funds_2024['index'] == '否')  # remove index funds
                                & (data_funds_2024['fund_inception_date'] < start_2024)  # remove funds start after 2024-01-01
                                & (data_funds_2024['fee_redemption_max '] != "--")  # drop missing values
                                & (data_funds_2024['fee_subscription_max'] != "--")  # drop missing values
                                & (data_funds_2024['fee_custodian'] != "--")  # drop missing values
                                & (data_funds_2024['fee_management'] != "--")  # drop missing values
                                & (data_funds_2024['ror_2024'] != '--')  # drop missing values
                                & (data_funds_2024['nav'] != 0)  # remove funds with NAV=0
                                ]

In [13]:
# replace "--" with 0 in percents of asset allocation
data_funds_2024.loc[:, 'pct_stock_2024'] = [0 if i == '--' else i for i in data_funds_2024['pct_stock_2024']]
data_funds_2024.loc[:, 'pct_bond_2024'] = [0 if i == '--' else i for i in data_funds_2024['pct_bond_2024']]
data_funds_2024.loc[:, 'pct_fund_2024'] = [0 if i == '--' else i for i in data_funds_2024['pct_fund_2024']]
data_funds_2024.loc[:, 'pct_mm_2024'] = [0 if i == '--' else i for i in data_funds_2024['pct_mm_2024']]
data_funds_2024.loc[:, 'pct_deposit_2024'] = [0 if i == '--' else i for i in data_funds_2024['pct_deposit_2024']]

# data type to float
data_funds_2024[['pct_stock_2024', 'pct_bond_2024', 'pct_fund_2024', 'pct_mm_2024', 'pct_deposit_2024']] = data_funds_2024[['pct_stock_2024', 'pct_bond_2024', 'pct_fund_2024', 'pct_mm_2024', 'pct_deposit_2024']].astype('float32')

In [14]:
# descriptive statistics
data_funds_2024[['pct_stock_2024', 'pct_bond_2024', 'pct_fund_2024', 'pct_mm_2024', 'pct_deposit_2024']].describe()

,pct_stock_2024,pct_bond_2024,pct_fund_2024,pct_mm_2024,pct_deposit_2024
count,10474.000000,10474.000000,10474.000000,10474.000000,10474.000000
mean,44.543896,51.745144,0.271056,0.002850,9.945718
std,40.764629,52.945480,4.669264,0.206223,91.990105
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.950025
50%,29.904600,34.057650,0.000000,0.000000,5.444500
75%,88.279198,102.737051,0.000000,0.000000,11.468350
max,99.528999,189.760193,94.716301,14.925900,6614.346680


各类资产收益率
   1. 债券：CBA00101.CB[中债新综合指数(总值)财富指数]：2024年7.60%
   2. 股票：700001.TI[同花顺全A(加权)]：2024年10.29%
   3. 基金：H11020.CSI[中证基金]：2024年4.93%
   4. MMF：H11025.CSI[货币基金]：2024年1.75%
   5. 存款：活期存款利率：0.2% （2024-10-18后调整为0.1%）

In [15]:
# Calculate benchmark, excess return, total fee

# rate of return of different assets in 2024
# data sourced from iFind
stock_r_24 = 0.1029
bond_r_24 = 0.0760
fund_r_24 = 0.0493
mmf_r_24 = 0.0175
deposit_r_24 = 0.0020

# define a function to calculate benchmark
def cal_benchmark(i):
    return i.iloc[0] * stock_r_24 + i.iloc[1] * bond_r_24 + i.iloc[2] * fund_r_24 + i.iloc[3] * mmf_r_24 + i.iloc[4] * deposit_r_24

# apply the function to data (%)
data_funds_2024['benchmark_rate'] = data_funds_2024[['pct_stock_2024', 'pct_bond_2024', 'pct_fund_2024', 'pct_mm_2024', 'pct_deposit_2024']].apply(cal_benchmark, axis=1)

# calculate the excess return (%)
data_funds_2024['excess_return'] = data_funds_2024['ror_2024'] - data_funds_2024['benchmark_rate']

# calculate total fee (max, %)
data_funds_2024['total_fee_max'] = data_funds_2024['fee_custodian'] + data_funds_2024['fee_management'] + data_funds_2024['fee_redemption_max '] + data_funds_2024['fee_subscription_max']

In [16]:
# calculate NAV-weighted sum as we need to calculate weighted average later
data_funds_2024['weighted_sum_benchmark_rate'] = data_funds_2024['benchmark_rate'] * data_funds_2024['nav']
data_funds_2024['weighted_sum_excess_return'] = data_funds_2024['excess_return'] * data_funds_2024['nav']
data_funds_2024['weighted_sum_fee_total'] = data_funds_2024['total_fee_max'] * data_funds_2024['nav']

In [17]:
# reset index and save processed data
data_funds_2024.reset_index(inplace=True, drop=True)
data_funds_2024.to_csv(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Processed\ADA_Data_Fund_2024.csv", encoding='gbk')

#### Merging Data
merge data of funds, fund managers, fund companies

In [18]:
# merge data of managers and funds
# on=['fund_code', 'manager_name']
data_funds_1 = pd.merge(data_funds_2024, data_managers_output, on=['fund_code', 'manager_name'], how='inner') 

In [19]:
# merge again with data of companies
data_funds_2 = pd.merge(data_funds_1, data_companies_output, on='company_name_short', how='inner')

In [20]:
# select and rearrange columns
# managers -> companies -> funds
data_funds_3 = data_funds_2[['manager_name', 'female', 'manager_edu_num',
       'experience_days', 'aum_manager', 'num_fund_manager_curr',
       'num_fund_manager_hist', 'company_name_short', 'company_name', 'company_rank_aum',
       'aum_non_mmf', 'aum', 'num_funds', 'num_managers', 'avg_aum_manager',
       'avg_num_manager', 'company_duration', 'fund_code', 'fund_name', 
       'manager_days', 'fof', 'index', 'category_l1', 'category_l2', 'nav',
       'fee_management', 'fee_custodian', 'fee_subscription_max',
       'fee_redemption_max ', 'total_fee_max', 'ror_2024', 'pct_stock_2024', 'pct_bond_2024',
       'pct_fund_2024', 'pct_mm_2024', 'pct_deposit_2024',
       'fund_inception_date', 'benchmark_rate', 'excess_return',
       'weighted_sum_benchmark_rate', 'weighted_sum_excess_return', 'weighted_sum_fee_total']]

In [21]:
# save processed data (pre-grouped) (Version 1)
data_funds_3.to_csv(r'c:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Processed\ADA_Data_V1.csv', encoding='gbk')

#### Manager-Level Data

In [22]:
# group data by managers
# to avoid Duplicate names, also by companies
grouped = data_funds_3.groupby(['manager_name', 'company_name'])

In [23]:
# calculate NAV-weighted averages
# recall that we calculate NAV-weighted sum already
def weighted_average_b(group):  # AV-weighted average benchmark rate
    return (group['weighted_sum_benchmark_rate'].sum() / group['nav'].sum())

def weighted_average_e(group):  # AV-weighted average excess return
    return (group['weighted_sum_excess_return'].sum() / group['nav'].sum())

def weighted_average_f(group):  # AV-weighted average total fee rate
    return (group['weighted_sum_fee_total'].sum() / group['nav'].sum())

weighted_average_benchmark_rate = grouped.apply(weighted_average_b, include_groups=False)
weighted_average_excess_return = grouped.apply(weighted_average_e, include_groups=False)
weighted_average_fee_total = grouped.apply(weighted_average_f, include_groups=False)

In [24]:
# drop duplicates to get manager-level data
# set multi-index
data_funds_4 = data_funds_3.drop_duplicates(subset=['manager_name', 'company_name']).reset_index(drop=True).set_index(['manager_name', 'company_name'])

In [25]:
# select and rearrange
data_funds_4 = data_funds_4[['female', 'manager_edu_num',
       'experience_days', 'aum_manager', 'num_fund_manager_curr',
       'num_fund_manager_hist', 'company_name_short', 'company_rank_aum',
       'aum_non_mmf', 'aum', 'num_funds', 'num_managers', 'avg_aum_manager',
       'avg_num_manager', 'company_duration']]

# insert weighted averages
data_funds_4['weighted_average_benchmark_rate'] = weighted_average_benchmark_rate
data_funds_4['weighted_average_excess_return'] = weighted_average_excess_return
data_funds_4['weighted_average_fee_total'] = weighted_average_fee_total

In [26]:
# save processed data (grouped) (Version 2)
data_funds_4.to_csv(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Processed\ADA_Data_V2.csv", encoding='gbk')

#### Facial Attractiveness Scores

In [27]:
# pictures from Asset Management Association of China (AMAC): https://gs.amac.org.cn/amac-infodisc/res/pof/person/personOrgList.html
# Scored on FacePlusPlus API
# simple processing with Excel
data_facial = pd.read_csv(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\ADA_Data_Fund_Manager_Scores.csv", encoding='utf-8', index_col=False).drop('Unnamed: 0', axis=1)

#### Previous Experience

In [28]:
# load data of previous experience
# data sourced from the website of AMAC, crawling
# previous experience calculated as the latest creation date - the earliest creation date
data_pre_experience_raw = pd.read_excel(r"C:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\ADA_Data_Fund_Manager_Infos.xlsx")

In [29]:
# select and rearrange columns
data_pre_experience = data_pre_experience_raw[['manager_name', 'company_name', 'earliest', 'latest', 'experience_before']]

In [30]:
# merging
data_managers_ = pd.merge(data_facial, data_pre_experience, on=['manager_name', 'company_name'], how="inner")

In [31]:
# add scores and previous experience data
data_funds_5 = pd.merge(data_funds_4, data_managers_, on=['company_name', 'manager_name'], how='inner')

# sort and reset index 
data_funds_5.sort_values(by=['company_name', 'manager_name'], ascending=True, inplace=True)
data_funds_5.reset_index(inplace=True, drop=True)

In [32]:
# save processed data (with facial attractiveness scores) (Version 3)
data_funds_5.to_csv(r'c:\Users\lzc19\Desktop\PHBS 24-25 Fall II\ADA\Project\Data\Data_Processed\ADA_Data_V3.csv', encoding='gbk')